In [1]:
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

[nltk_data] Downloading package punkt to /home/mohammad/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
import numpy
import random 
import tensorflow as tf
import json 
#impoort the split 
from sklearn.model_selection import train_test_split
import numpy as np
#import keras classifier   
#IMPORY SGD 
from tensorflow.keras.optimizers import SGD


In [7]:

with open("/home/mohammad/Desktop/Firs_AId-1.0/intents (2).json") as file:
    data = json.load(file)   


In [8]:
import numpy as np

# Define preprocessing functions
def tokenize(sentence):
    return nltk.word_tokenize(sentence)

def stem(word):
    return stemmer.stem(word.lower())

# Initialize lists for words, labels, training data, and output
words = []
labels = []
training_data = []
output_data = []

# Tokenize and stem each pattern, and prepare training data and output
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        tokens = tokenize(pattern)
        words.extend(tokens)
        training_data.append(tokens)
        output_data.append(intent["tag"])

        if intent["tag"] not in labels:
            labels.append(intent["tag"])

# Stem and lowercase words, remove duplicates, and sort
words = sorted(list(set([stem(word) for word in words if word != "?"])))
labels = sorted(labels)

# Initialize empty output rows
out_empty = [0] * len(labels)

# Prepare training and output arrays
X = []
y = []

for idx, doc in enumerate(training_data):
    bag = []

    # Stem and lowercase words in the pattern
    tokens = [stem(word) for word in doc]

    # Create bag of words
    for word in words:
        bag.append(1) if word in tokens else bag.append(0)

    # Create output row
    output_row = list(out_empty)
    output_row[labels.index(output_data[idx])] = 1

    X.append(bag)
    y.append(output_row)

# Conve
training = np.array(X)
output = np.array(y)




In [11]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

# Reset the default graph
tf.reset_default_graph()

# Define the model
model = tf.keras.Sequential([
    # Input layer (implicit by specifying input_shape in the first Dense layer)
    tf.keras.layers.Dense(128, activation='relu', input_shape=(len(training[0]),)),  
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation='relu'),  
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(len(output[0]), activation='softmax')
])

# Define optimizer
sgd = tf.train.MomentumOptimizer(learning_rate=0.01, momentum=0.9, use_nesterov=True)

model.compile(optimizer=sgd,
              loss='categorical_crossentropy',
              metrics=['accuracy']
              )    

# Fit the model
model.fit(training, output, epochs=300, batch_size=5, verbose=1)


Instructions for updating:
non-resource variables are not supported in the long term


Instructions for updating:
non-resource variables are not supported in the long term


Train on 962 samples
Epoch 1/300
962/962 [==============================] - 0s 211us/sample - loss: 5.8471 - acc: 0.0322
Epoch 2/300
962/962 [==============================] - 0s 123us/sample - loss: 5.2055 - acc: 0.0457
Epoch 3/300
962/962 [==============================] - 0s 119us/sample - loss: 4.7249 - acc: 0.1320
Epoch 4/300
962/962 [==============================] - 0s 123us/sample - loss: 4.2927 - acc: 0.2069
Epoch 5/300
962/962 [==============================] - 0s 121us/sample - loss: 3.9845 - acc: 0.2412
Epoch 6/300
962/962 [==============================] - 0s 128us/sample - loss: 3.6656 - acc: 0.2775
Epoch 7/300
962/962 [==============================] - 0s 119us/sample - loss: 3.3477 - acc: 0.3264
Epoch 8/300
962/962 [==============================] - 0s 122us/sample - loss: 3.0768 - acc: 0.3680
Epoch 9/300
962/962 [==============================] - 0s 118us/sample - loss: 2.8211 - acc: 0.4116
Epoch 10/300
962/962 [==============================] - 0s 128us/sample - loss:

In [12]:
model 

In [13]:
def bag_of_words(s,words):
	bag = [0 for _ in range(len(words))]


	s_words = nltk.word_tokenize(s)
	s_words = [stemmer.stem(word.lower()) for word in s_words]

	for se in s_words:
		for i, w in enumerate(words):
			if w == se:
				bag[i] = 1

	return numpy.array(bag)

In [14]:
def chat():
    print("Start Talking with the bot (type quit to stop)")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        # Preprocess input text into bag of words
        inp_bag_of_words = bag_of_words(inp, words)

        # Reshape input for model prediction
        inp_bag_of_words = inp_bag_of_words.reshape(1, inp_bag_of_words.shape[0])

        # Predict using the model
        results = model.predict(inp_bag_of_words)[0]

        results_index = numpy.argmax(results)
        tag = labels[results_index]

        if results[results_index] > 0.5:
            for tg in data["intents"]:
                if tg['tag'] == tag:
                    responses = tg['responses']
            print(random.choice(responses))
            print("\n")

        else:
            print("I didn't get that, try again")


In [15]:
chat()

Start Talking with the bot (type quit to stop)


/home/mohammad/.local/lib/python3.8/site-packages/keras/src/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


Wash the cut properly to prevent infection and stop the bleeding by applying pressure for 1-2minutes until bleeding stops. Apply Petroleum Jelly to make sure that the wound is moist for quick healing. Finally cover the cut with a sterile bandage. Pain relievers such as acetaminophen can be applied.


Wash the cut properly to prevent infection and stop the bleeding by applying pressure for 1-2minutes until bleeding stops. Apply Petroleum Jelly to make sure that the wound is moist for quick healing. Finally cover the cut with a sterile bandage. Pain relievers such as acetaminophen can be applied.


For eye injuries, do not try to treat severe injuries yourself. Avoid touching, rubbing, or applying pressure to the eye. Do not attempt to remove objects stuck in the eye or wash out the eye for chemical exposures or punctures. Cover the eye with a sterile dressing that does not make contact with the eye itself. Seek immediate medical attention, especially if the injury involves chemicals, sh